In [19]:
import pandas as pd

In [99]:
df = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9/last_contribution/openj9_dataset_6_7_24_last_contrib.csv")

In [100]:
df2 = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/data/openj9_dataset_6_7_24_last.csv")

In [101]:
df["assignees"] = df.assignees.str.lower()

In [102]:
list(df.assignees.values) == list(df2.owner.values)

False

In [103]:
df2[df2["issue_number"] == 15306]

,issue_number,issue_title,issue_body,issue_url,issue_state,creator,labels,owner,component
6027,15306,JDK8 OpenJ9 Gives Different Results from HotSp...,Java version\r\n-------------\r\n\r\n```\r\nop...,https://github.com/eclipse-openj9/openj9/issue...,closed,connglli,"comp:jit, userRaised",0xdaryl,comp:jit
6028,15306,JDK8 OpenJ9 Gives Different Results from HotSp...,Java version\r\n-------------\r\n\r\n```\r\nop...,https://github.com/eclipse-openj9/openj9/issue...,closed,connglli,"comp:jit, userRaised",hzongaro,comp:jit


In [106]:
df2.owner.unique()

array(['gireeshpunathil', 'mpirvu', 'r30shah', 'hzongaro', 'danheidinga',
       'tajila', 'smlambert', 'chetanmeh', 'dhong44', 'hangshao0',
       'ailloviee', 'renfeiw', 'cathyzhyi', 'fjeremic', nan, 'ebadyano',
       'keithc-ca', 'jameskingdon', 'chengjin01', 'adambrousseau',
       'mstoodle', 'chandrams', 'vsebe', 'dsouzai', 'fengxue-is',
       'pdbain-ibm', 'sophia-guo', 'terryzuoty', 'llxia', 'babsingh',
       'amicic', 'mikezhang1234567890', 'alexey-anufriev', 'pshipton',
       '0dvictor', 'harryyu1994', 'adamfarley', 'yathamravali',
       'jasonfengj9', 'a7ehuo', 'cedrichansen', 'dmitripivkine',
       'andrewcraik', 'dchopra001', 'kdbeall', 'yanluo7', 'yuehan-lin',
       'alenbadel', 'hanfeiyu', 'lumpfish', '0xdaryl', 'pdbain', 'liqunl',
       'rbanerjee', 'knn-k', 'gacholio', 'andrew-m-leonard',
       'scott-young-6746', 'theresa-m', 'nbhuiyan', 'joransiu',
       'non-conformances-research', 'dnakamura', 'mesbah-alam',
       'cwesmills', 'suechaplain', 'linhu2016',

In [107]:
# Define active user map
vm_users = [
    "pshipton",
    "keithc-ca",
    "gacholio",
    "tajila",
    "babsingh",
    "JasonFengJ9",
    "fengxue-IS",
    "hangshao0",
    "theresa-m",
    "ChengJin01",
    "singh264",
    "thallium",
    "ThanHenderson",
]
jvmti_users = ["gacholio", "tajila", "babsingh", "fengxue-IS"]
jclextensions_users = ["JasonFengJ9", "pshipton", "keithc-ca"]
test_users = ["LongyuZhang", "annaibm", "sophiaxu0424", "KapilPowar", "llxia"]
build_users = ["adambrousseau", "mahdipub"]
gc_users = ["dmitripivkine", "amicic", "kangyining", "LinHu2016"]

# Putting them in dictionaries
components = {
    "comp:vm": vm_users,
    "comp:jvmti": jvmti_users,
    "comp:jclextensions": jclextensions_users,
    "comp:test": test_users,
    "comp:build": build_users,
    "comp:gc": gc_users,
}

expected_users = [user.lower() for user_list in components.values() for user in user_list]
df2 = df2[df2["owner"].isin(expected_users)]
print(f"Total issues after developer filtering: {len(df2)}")

df2 = df2.sort_values(by="issue_number")

Total issues after developer filtering: 2220


In [108]:
xx = set(expected_users)

In [109]:
yy = set(df2.owner.unique())

In [110]:
xx - yy

{'kapilpowar', 'mahdipub'}

In [111]:
df2.owner.value_counts()

owner
jasonfengj9      377
pshipton         329
keithc-ca        277
babsingh         262
llxia            111
hangshao0        111
gacholio         107
fengxue-is       106
chengjin01       101
tajila            89
adambrousseau     71
theresa-m         62
longyuzhang       42
linhu2016         39
dmitripivkine     34
thallium          30
amicic            27
singh264          24
thanhenderson     13
kangyining         6
sophiaxu0424       1
annaibm            1
Name: count, dtype: int64

In [112]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df2, test_size=0.1, shuffle=False)

sample_threshold = 20
developers = df_train["owner"].value_counts()
filtered_developers = developers.index[developers >= sample_threshold]
df_train = df_train[df_train["owner"].isin(filtered_developers)]

train_owners = set(df_train["owner"])
test_owners = set(df_test["owner"])

# unwanted = list(test_owners - train_owners)

# df_test = df_test[~df_test["assignees"].isin(unwanted)]

print(f"Training data: {len(df_train)}, Validation data: {len(df_test)}")
print(f"Number of developers: {len(df_train.owner.unique())}")

print(f"Train dataset size: {len(df_train)}")
print(f"Test dataset size: {len(df_test)}")

Training data: 1975, Validation data: 222
Number of developers: 17
Train dataset size: 1975
Test dataset size: 222


In [20]:
gg = pd.read_csv("/home/mdafifal.mamun/notebooks/triagerX/openj9_train_17_devs.csv")

In [21]:
gg = gg[gg["owner"].notna()]

In [35]:
gggg = {}

for _, r in gg.iterrows():
    gggg[r["owner"]] = r["owner_id"]

gggg = dict(sorted(gggg.items(), key=lambda item: item[1]))

print(gggg)

{'adambrousseau': 0, 'amicic': 1, 'babsingh': 2, 'chengjin01': 3, 'dmitripivkine': 4, 'fengxue-is': 5, 'gacholio': 6, 'hangshao0': 7, 'jasonfengj9': 8, 'keithc-ca': 9, 'linhu2016': 10, 'llxia': 11, 'longyuzhang': 12, 'pshipton': 13, 'tajila': 14, 'thallium': 15, 'theresa-m': 16}


In [17]:
sample_threshold = 1
developers = gg["owner"].value_counts()
filtered_developers = developers.index[developers >= sample_threshold]
gg = gg[gg["owner"].isin(filtered_developers)]

In [36]:
import yaml

In [37]:
with open("/home/mdafifal.mamun/notebooks/triagerX/app/config/triagerx_config.yaml") as f:
    cc = yaml.safe_load(f)

In [42]:
set(cc["developer_id_map"].keys())

{'adambrousseau',
 'amicic',
 'babsingh',
 'chengjin01',
 'dmitripivkine',
 'fengxue-is',
 'gacholio',
 'hangshao0',
 'jasonfengj9',
 'keithc-ca',
 'linhu2016',
 'llxia',
 'longyuzhang',
 'pshipton',
 'tajila',
 'thallium',
 'theresa-m'}

In [18]:
len(gg)

2931

In [7]:
len(gg.owner.unique())

18

In [55]:
print(gg.iloc[17].issue_body)

### Description of the new feature / enhancement

PowerToys is an incubation system for testing new work.  Some features are on be default, others off.  Most of the time, we will default them to on.  This, from time to time, leads to unexpected results when someone quickly invokes find my mouse or other utilities. 

My proposal is to have a more upfront system in both first time run / upgrade as well as the 1st page in settings to show crisply, 

1. if a utility is on
2. Invoke shortcut
3. Shortcut to the utility's settings

When new utilities come in, this line would also be added to the top of the upgraded dialog as well.

### Scenario when this would be used?

Getting new users and upgrade paths crisply aware of new things and defaults

# Concept
This is a very conceptual and quick-and-dirty comp of what a new homepage experience could look like:

![image](https://github.com/microsoft/PowerToys/assets/9866362/1e2676c8-9c2a-4897-9c14-77ea13eaa684)

- It would have a list of all modul

In [30]:
import re

def clean_text(text: str) -> str:
        text = str(text)  # In case, there is nan or something else
        cleaned_text = text.strip()

        cleaned_text = re.sub(r"(https?|ftp):\/\/[^\s/$.?#].[^\s]*", "", cleaned_text)
        cleaned_text = re.sub(r"0x[\da-fA-F]+", "", cleaned_text)
        cleaned_text = re.sub(r"\b[0-9a-fA-F]{16}\b", "", cleaned_text)
        cleaned_text = re.sub(
            r"\b\d{2}:\d{2}:\d{2}:\d{4,} GMT\b",
            "",
            cleaned_text,
        )
        cleaned_text = re.sub(
            r"\b\d{2}:\d{2}:\d{2}(\.\d{2,3})?\b",
            "",
            cleaned_text,
        )
        cleaned_text = re.sub(
            r"\b\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+Z\b",
            "",
            cleaned_text,
        )
        cleaned_text = re.sub(r"```", "", cleaned_text)
        # cleaned_text = re.sub(r"-{3,}", "", cleaned_text)
        cleaned_text = re.sub(r"[\*#=+\-]{3,}", "", cleaned_text)

        cleaned_text = re.sub(r"(\r?\n)+", "\n", cleaned_text)
        cleaned_text = re.sub(r"(?![\r\n])\s+", " ", cleaned_text)
        cleaned_text = cleaned_text.strip()

        return cleaned_text

In [31]:
gg.iloc[17].issue_body

'### Description of the new feature / enhancement\r\n\r\nhttps://github.com/microsoft/PowerToys/blob/77063e0767a0619abfc339db3a2a564d27310b6c/src/modules/launcher/Plugins/Microsoft.Plugin.Shell/Main.cs#L51-L62\r\nCurrently, the shell plugin in the powertoys can not use `pwsh.exe` to execute shell, I consider we should support it.\r\n\r\n### Scenario when this would be used?\r\n\r\nwe can use powershell 7 to instead the powershell 5 which the system provides. \r\n\r\n### Supporting information\r\n\r\n_No response_\r\n'

In [32]:
print(clean_text(gg.iloc[17].issue_body))

Description of the new feature / enhancement
Currently, the shell plugin in the powertoys can not use `pwsh.exe` to execute shell, I consider we should support it.
 Scenario when this would be used?
we can use powershell 7 to instead the powershell 5 which the system provides. Supporting information
_No response_


In [39]:
from transformers import AutoTokenizer

In [77]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-large")

In [98]:
tokenizer.decode([38500])

'554'

In [94]:
tokenizer("1582275888554")

{'input_ids': [1, 26758, 2036, 2545, 22410, 38500, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [46]:
special_tokens = {"additional_special_tokens": ["<time>"]}
tokenizer.add_special_tokens(special_tokens)

tokenizer.add_special_tokens(special_tokens)

0

In [52]:
tokenizer("01:40:14")

{'input_ids': [1, 2663, 35, 1749, 35, 1570, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [1]:
from triagerx.model.module_factory import DatasetFactory, ModelFactory

In [24]:
gg = ModelFactory.get_model(
    model_key="triagerx",
    output_size=1032,
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=["microsoft/deberta-base", "roberta-base"],
    dropout=0.2,
    max_tokens=256,
    label_map={},
)

2024-06-24 16:16:51.810 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.triagerx_dev_model.TriagerxDevModel'>
2024-06-24 16:16:51.811 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers
2024-06-24 16:16:51.811 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
yy = ModelFactory.get_model(
    model_key="cnn-transformer",
    output_size=1032,
    unfrozen_layers=3,
    num_classifiers=3,
    base_models=["microsoft/deberta-large"],
    dropout=0.2,
    max_tokens=256,
    label_map={},
)

2024-06-24 16:17:01.150 | DEBUG    | triagerx.model.module_factory:get_model:54 - Instantiating model of class: <class 'triagerx.model.cnn_transformer.CNNTransformer'>
2024-06-24 16:17:01.151 | DEBUG    | triagerx.model.module_factory:get_model:65 - Including number of filters and classifiers


In [33]:
total_params = sum(p.numel() for p in gg.parameters())

total_params2 = sum(p.numel() for p in yy.parameters())

In [34]:
total_params - total_params2

-134837751

In [35]:
print(total_params, total_params2)

287663649 422501400


In [37]:
(total_params2 - total_params) / total_params2

0.3191415484067035

In [47]:
vv = {1: "sadasd", 2: "alksdj"}

*vv

SyntaxError: can't use starred expression here (383884787.py, line 3)